# **Feature Selection**: RF and Chi Squared
------------------

In [1]:
# https://infosecjupyterthon.com/notebooks/2_Jupyterthon_Cyb3rPandaH_2020.html
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.functions import length, concat ,concat_ws

import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
import requests
from openhunt import ossem, descriptiveStatistics as ds, visualizations as vis
import warnings
warnings.filterwarnings('ignore')

# Import packages
import os
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
%matplotlib inline

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")
spark.sparkContext.setCheckpointDir("./tmp/checkpoints")
spark.sparkContext.version
spark.conf.set("spark.driver.memory", "3g")
spark.conf.set("spark.executor.memory", "3g")

In [4]:
df_index = spark.read.json('df_labelled_coalesce.json')

In [5]:
df_index = df_index.select("*").withColumn("id", func.monotonically_increasing_id()).drop("tags")

In [6]:
df_index = df_index.withColumn("TP_Binary", func.when((df_index["TP_Label"]=='0'), 0).otherwise(1))

In [6]:
df_index = df_index.checkpoint()

In [7]:
df_index = df_index.cache()

In [8]:
# Try to Pands thru Spark -> post checkpoint + cache -> : java.lang.OutOfMemoryError: Java heap space -> No change
# pd_index = df_index.toPandas()

In [7]:
df_index.createTempView('apt29Host')

In [8]:
target_host_day1 = spark.sql(
'''
SELECT *
FROM apt29Host
WHERE NOT TP_Label = '0'
''')

In [11]:
pd_target_distict_count = target_host_day1.agg(*[
    func.countDistinct(c).alias(c)    # vertical (column-wise) operations in SQL ignore NULLs
    for c in target_host_day1.columns
]).toPandas().transpose()

In [59]:
pd.options.display.max_rows = 100
pd_target_distict_count.sort_values(by = 0, ascending = False).head(30)

,0
id,3438
RecordNumber,759
Message,756
ActivityID,638
ContextInfo,623
@timestamp,475
Payload,416
EventReceivedTime,87
EventTime,78
UtcTime,59


In [57]:
# Select subset of columns with more than 1 AND less than 52 unique values
columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:3].index.values)

In [58]:
columnList

['ProcessId', 'ProcessGuid', 'ThreadID']

In [23]:
#columnList = ["EventID", "ProcessGuid"]

In [14]:
# Class Balance
df_index.groupBy(df_index["TP_Binary"]).count().orderBy('count', ascending=False).toPandas()

,TP_Binary,count
0,0,195322
1,1,3438


In [15]:
# Categorical Freq Count by Columns with more than one distinct value in the target subset
#for i in columnList:
#    display(df_index.groupBy(df_index[i]).count().orderBy('count', ascending=False).toPandas())

,@timestamp,count
0,2020-05-02T03:00:36.941Z,121
1,2020-05-02T03:00:37.248Z,117
2,2020-05-02T03:20:38.217Z,99
3,2020-05-02T03:20:46.921Z,99
4,2020-05-02T03:20:53.888Z,98
...,...,...
50318,2020-05-02T03:08:18.613Z,1
50319,2020-05-02T03:01:35.720Z,1
50320,2020-05-02T02:56:17.115Z,1
50321,2020-05-02T03:03:07.031Z,1


,AccessList,count
0,None,187719
1,%%4432\r\n\t\t\t\t,6773
2,%%1538\r\n\t\t\t\t%%4432\r\n\t\t\t\t%%4435\r\n...,3821
3,%%1538\r\n\t\t\t\t%%1541\r\n\t\t\t\t%%4416\r\n...,44
4,%%1537\r\n\t\t\t\t%%1538\r\n\t\t\t\t%%4432\r\n...,44
5,%%4416\r\n\t\t\t\t,42
6,%%1538\r\n\t\t\t\t%%1541\r\n\t\t\t\t%%4416\r\n...,36
7,%%1537\r\n\t\t\t\t,30
8,%%1541\r\n\t\t\t\t%%4416\r\n\t\t\t\t%%4423\r\n...,30
9,%%1537\r\n\t\t\t\t%%1538\r\n\t\t\t\t%%1539\r\n...,26


,AccessMask,count
0,None,187664
1,0x1,6815
2,0x20019,3821
3,0x3001f,44
4,0x120089,44
5,0x12019f,36
6,0xf003f,36
7,0x2,36
8,0x100081,30
9,0x10000,30


,AccountName,count
0,SYSTEM,145230
1,None,46308
2,pbeesly,7158
3,NETWORK SERVICE,41
4,LOCAL SERVICE,23


,ActivityID,count
0,None,186806
1,{BCCD0404-4290-0001-60ED-CE673020D601},3177
2,{A725FCD6-CBDF-0000-6337-C31E1D20D601},202
3,{55C748F1-AE9F-0001-EE1C-BFD51F20D601},107
4,{7AEC1283-BA78-47DB-AB99-27A339F7971E},70
...,...,...
5218,{BCCD0404-4290-0000-CE1E-14D71F20D601},1
5219,{BCCD0404-4290-0001-E44A-13D71F20D601},1
5220,{A725FCD6-CBDF-0001-F27D-D81E1D20D601},1
5221,{BCCD0404-4290-0000-B1FC-10D71F20D601},1


...

,User,count
0,None,196663
1,NT AUTHORITY\SYSTEM,1174
2,DMEVALS\pbeesly,790
3,NT AUTHORITY\NETWORK SERVICE,75
4,NT AUTHORITY\LOCAL SERVICE,53
5,Window Manager\DWM-2,1
6,Window Manager\DWM-1,1
7,Font Driver Host\UMFD-1,1
8,Font Driver Host\UMFD-0,1
9,Font Driver Host\UMFD-2,1


,UserID,count
0,S-1-5-18,145230
1,None,46308
2,S-1-5-21-1830255721-3727074217-2423397540-1107,7158
3,S-1-5-20,41
4,S-1-5-19,23


,UtcTime,count
0,None,54870
1,2020-05-02 03:19:31.447,397
2,2020-05-02 03:19:30.337,396
3,2020-05-02 03:19:30.165,384
4,2020-05-02 03:19:30.259,383
...,...,...
16376,2020-05-02 03:15:32.261,1
16377,2020-05-02 03:23:09.379,1
16378,2020-05-02 03:11:00.001,1
16379,2020-05-02 03:02:46.460,1


,Version,count
0,2.0,81873
1,3.0,59944
2,0.0,26316
3,1.0,22949
4,NaN,5290
5,5.0,2179
6,4.0,209


,id,count
0,26,1
1,29,1
2,474,1
3,964,1
4,1677,1
...,...,...
198755,42949703422,1
198756,42949703509,1
198757,42949704151,1
198758,42949704368,1


In [60]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

In [61]:
pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])

In [62]:
mod = pipe.fit(df_index)

In [63]:
df2 = mod.transform(df_index)

In [64]:
# Failed -> : org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.OutOfMemoryError: Java heap space
# Worked on columnList = ["EventID", "ProcessGuid"] feature list
# Worked on columnList = ['ProcessId', 'ProcessGuid', 'ThreadID']
df2 = df2.checkpoint()

In [65]:
# Worked
df2 = df2.cache()

In [67]:
df2.printSchema()

root
 |-- @timestamp: string (nullable = true)
 |-- @version: string (nullable = true)
 |-- AccessList: string (nullable = true)
 |-- AccessMask: string (nullable = true)
 |-- AccessReason: string (nullable = true)
 |-- AccountName: string (nullable = true)
 |-- AccountType: string (nullable = true)
 |-- Action: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- ActiveProfile: string (nullable = true)
 |-- ActivityID: string (nullable = true)
 |-- AdapterName: string (nullable = true)
 |-- AdapterSuffixName: string (nullable = true)
 |-- AdditionalInfo: string (nullable = true)
 |-- AdditionalInfo2: string (nullable = true)
 |-- AdvancedOptions: string (nullable = true)
 |-- AlgorithmName: string (nullable = true)
 |-- Application: string (nullable = true)
 |-- Archived: string (nullable = true)
 |-- AuthenticationPackageName: string (nullable = true)
 |-- BitlockerUserInputTime: string (nullable = true)
 |-- BootMenuPolicy: string (nullable = true)
 |-- BootMode: stri

In [68]:
mod.stages[-1].featureImportances

SparseVector(2015, {1: 0.2435, 17: 0.1124, 24: 0.0235, 698: 0.208, 1113: 0.1131, 1777: 0.06, 1921: 0.2395})

In [69]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [70]:
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(10)

,idx,name,score
1,1,OHE_ProcessId_5044,0.243494
1921,1921,OHE_ThreadID_8416,0.239530
698,698,OHE_ProcessId_0x87c,0.207961
1113,1113,OHE_ProcessGuid_{47ab858c-dac2-5eac-e802-00000...,0.113114
17,17,OHE_ProcessId_1960,0.112398
1777,1777,OHE_ThreadID_1740,0.060042
24,24,OHE_ProcessId_5360,0.023460
1342,1342,OHE_ProcessGuid_{5aa8ec29-cae5-5eac-5d00-00000...,0.000000
1343,1343,OHE_ProcessGuid_{47ab858c-e378-5eac-db03-00000...,0.000000
1344,1344,OHE_ProcessGuid_{47ab858c-e6b2-5eac-5600-00000...,0.000000


In [41]:
# Retry with more columns

In [71]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 5
columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:5].index.values)

In [72]:
# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

In [73]:
pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])

In [74]:
mod = pipe.fit(df_index)

In [75]:
df2 = mod.transform(df_index)

In [76]:
mod.stages[-1].featureImportances

SparseVector(4176, {1: 0.0981, 5: 0.0106, 11: 0.0119, 20: 0.0107, 45: 0.0047, 72: 0.0044, 84: 0.0024, 208: 0.034, 1112: 0.0156, 1758: 0.0777, 1761: 0.0282, 1763: 0.0289, 1835: 0.0061, 1921: 0.0291, 2075: 0.0065, 2291: 0.0199, 2300: 0.1385, 2337: 0.0505, 2380: 0.0901, 2414: 0.0505, 2422: 0.1088, 2428: 0.1429, 3455: 0.03})

In [77]:
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(10)

,idx,name,score
2428,2428,OHE_NewProcessId_0x2290,0.142857
2300,2300,OHE_NewProcessId_0xef8,0.138478
2422,2422,OHE_NewProcessId_0x102c,0.108820
1,1,OHE_ProcessId_5044,0.098140
2380,2380,OHE_NewProcessId_0x195c,0.090089
1758,1758,OHE_ThreadID_4588,0.077664
2414,2414,OHE_NewProcessId_0xf0c,0.050453
2337,2337,OHE_NewProcessId_0xbc,0.050451
208,208,OHE_ProcessId_8524,0.034037
3455,3455,OHE_Hashes_SHA1=61A5FFA57DFE0D9B3E4938439FF345...,0.030030


In [78]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 10
columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:10].index.values)

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(20)

,idx,name,score
4628,4628,OHE_Task_106,0.123079
1,1,OHE_ProcessId_5044,0.120104
4623,4623,OHE_Task_12,0.094825
2558,2558,OHE_Hashes_SHA1=36C5D12033B2EAF251BAE61C00690F...,0.086222
2711,2711,OHE_Hashes_SHA1=585EB59D12A111E9291518C5CF5D3F...,0.075562
4176,4176,OHE_Image_C:\windows\system32\svchost.exe,0.067394
4381,4381,OHE_ParentProcessGuid_{47ab858c-e43f-5eac-eb03...,0.063470
4626,4626,OHE_Task_7,0.059324
2897,2897,OHE_Hashes_SHA1=388B289E2FD96234E2C1E8AE777248...,0.042146
4508,4508,OHE_ParentProcessId_3480,0.038778


In [80]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)

pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, rf])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(20)

,idx,name,score
5067,5067,"OHE_CommandLine_""PowerShell.exe"" -noni -noexit...",0.137528
7621,7621,OHE_TargetObject_HKU\S-1-5-21-1830255721-37270...,0.120860
4865,4865,"OHE_CommandLine_""C:\Windows\Temp\python.exe""",0.103400
4875,4875,OHE_CommandLine_powershell,0.085716
4897,4897,"OHE_CommandLine_""C:\windows\system32\sdclt.exe""",0.081018
4432,4432,OHE_ParentProcessGuid_{47ab858c-e1e3-5eac-b603...,0.065404
1921,1921,OHE_ThreadID_8416,0.061839
5013,5013,"OHE_CommandLine_""C:\ProgramData\victim\â€®cod....",0.057142
5307,5307,OHE_ScriptBlockId_63fc6cf4-cd9f-4134-9231-51cc...,0.056115
2558,2558,OHE_Hashes_SHA1=36C5D12033B2EAF251BAE61C00690F...,0.055759


In [82]:
#pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100
pd.get_option("display.max_colwidth")

100

In [83]:
ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(20)

,idx,name,score
5067,5067,"OHE_CommandLine_""PowerShell.exe"" -noni -noexit -ep bypass -window hidden -c ""sal a New-Object;Ad...",0.137528
7621,7621,OHE_TargetObject_HKU\S-1-5-21-1830255721-3727074217-2423397540-1107\Software\Sysinternals\SDelet...,0.120860
4865,4865,"OHE_CommandLine_""C:\Windows\Temp\python.exe""",0.103400
4875,4875,OHE_CommandLine_powershell,0.085716
4897,4897,"OHE_CommandLine_""C:\windows\system32\sdclt.exe""",0.081018
4432,4432,OHE_ParentProcessGuid_{47ab858c-e1e3-5eac-b603-000000000400},0.065404
1921,1921,OHE_ThreadID_8416,0.061839
5013,5013,"OHE_CommandLine_""C:\ProgramData\victim\â€®cod.3aka3.scr"" /S",0.057142
5307,5307,OHE_ScriptBlockId_63fc6cf4-cd9f-4134-9231-51ccb5c7d247,0.056115
2558,2558,"OHE_Hashes_SHA1=36C5D12033B2EAF251BAE61C00690FFB17FDDC87,MD5=CDA48FC75952AD12D99E526D0B6BF70A,SH...",0.055759


In [84]:
columnList

['ProcessId',
 'ProcessGuid',
 'ThreadID',
 'NewProcessId',
 'Hashes',
 'Image',
 'ParentProcessGuid',
 'ParentProcessId',
 'Task',
 'EventID',
 'CommandLine',
 'ScriptBlockId',
 'RelativeTargetName',
 'TargetFilename',
 'ScriptBlockText',
 'TargetObject',
 'ExecutionProcessID',
 'ParentCommandLine',
 'SubjectLogonId',
 'Description']

In [101]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
#columnList = ["EventID", "ProcessGuid"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")

In [102]:
pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes])
mod = pipe.fit(df_index)
df2 = mod.transform(df_index)

In [103]:
from pyspark.ml.feature import ChiSqSelector
selector = ChiSqSelector(numTopFeatures=10, featuresCol='features', outputCol="selectedFeatures", labelCol='label', selectorType='numTopFeatures', percentile=0.1, fpr=0.05, fdr=0.05, fwe=0.05)

In [104]:
mod = selector.fit(df2)

In [105]:
result = mod.transform(df2)

In [108]:
mod.selectedFeatures

[0, 1, 2, 3, 4, 5, 1107, 1108, 1757, 1758]

In [112]:
result.head()

Row(@timestamp='2020-05-02T03:20:47.897Z', @version='1', AccessList=None, AccessMask=None, AccessReason=None, AccountName=None, AccountType=None, Action=None, Active=None, ActiveProfile=None, ActivityID=None, AdapterName=None, AdapterSuffixName=None, AdditionalInfo=None, AdditionalInfo2=None, AdvancedOptions=None, AlgorithmName=None, Application=None, Archived=None, AuthenticationPackageName=None, BitlockerUserInputTime=None, BootMenuPolicy=None, BootMode=None, BootStatusPolicy=None, BootType=None, BuildVersion=None, CallTrace=None, CallerProcessId=None, CallerProcessName=None, CalloutId=None, CalloutKey=None, CalloutName=None, CalloutType=None, Category='Kerberos Service Ticket Operations', ChangeType=None, Channel='Security', ClassId=None, ClassName=None, ClientCreationTime=None, ClientProcessId=None, CommandLine=None, Company=None, CompatibleIds=None, Conditions=None, Config=None, ConfigAccessPolicy=None, ContextInfo=None, CorruptionActionState=None, CountNew=None, CountOfCredential

In [110]:
result.head().selectedFeatures

SparseVector(10, {})

In [111]:
result.printSchema()

root
 |-- @timestamp: string (nullable = true)
 |-- @version: string (nullable = true)
 |-- AccessList: string (nullable = true)
 |-- AccessMask: string (nullable = true)
 |-- AccessReason: string (nullable = true)
 |-- AccountName: string (nullable = true)
 |-- AccountType: string (nullable = true)
 |-- Action: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- ActiveProfile: string (nullable = true)
 |-- ActivityID: string (nullable = true)
 |-- AdapterName: string (nullable = true)
 |-- AdapterSuffixName: string (nullable = true)
 |-- AdditionalInfo: string (nullable = true)
 |-- AdditionalInfo2: string (nullable = true)
 |-- AdvancedOptions: string (nullable = true)
 |-- AlgorithmName: string (nullable = true)
 |-- Application: string (nullable = true)
 |-- Archived: string (nullable = true)
 |-- AuthenticationPackageName: string (nullable = true)
 |-- BitlockerUserInputTime: string (nullable = true)
 |-- BootMenuPolicy: string (nullable = true)
 |-- BootMode: stri

In [113]:
r = ChiSquareTest.test(df2, "features", "label")

In [114]:
r

DataFrame[pValues: vector, degreesOfFreedom: array<int>, statistics: vector]

In [116]:
ChiSquareResults = []
for idx, result in enumerate(r):
    row = {
        'feature_index': idx,
        'p_value': result.pValue,
        'statistic': result.statistic,
        'degrees_of_freedom': result.degreesOfFreedom
    }
    ChiSquareResults.append(row)

In [117]:
ChiSquareResults

[{'feature_index': 0,
  'p_value': Column<b'pValues[pValue]'>,
  'statistic': Column<b'pValues[statistic]'>,
  'degrees_of_freedom': Column<b'pValues[degreesOfFreedom]'>},
 {'feature_index': 1,
  'p_value': Column<b'degreesOfFreedom[pValue]'>,
  'statistic': Column<b'degreesOfFreedom[statistic]'>,
  'degrees_of_freedom': Column<b'degreesOfFreedom[degreesOfFreedom]'>},
 {'feature_index': 2,
  'p_value': Column<b'statistics[pValue]'>,
  'statistic': Column<b'statistics[statistic]'>,
  'degrees_of_freedom': Column<b'statistics[degreesOfFreedom]'>}]

In [121]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
#columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
columnList = ["EventID", "ProcessGuid"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
selector = ChiSqSelector(numTopFeatures=50, featuresCol='features', outputCol="selectedFeatures", labelCol='label', selectorType='numTopFeatures', percentile=0.1, fpr=0.05, fdr=0.05, fwe=0.05)

In [122]:
pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, selector])

In [123]:
mod = pipe.fit(df_index)

In [124]:
df2 = mod.transform(df_index)

In [164]:
def ExtractFeatureSelection(featuresSelected, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
      list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(featuresSelected).rename(columns={0:"idx"})
    pd_list_extract = pd.DataFrame(list_extract)
    varlist['name'] = varlist['idx'].apply(lambda x: pd_list_extract.iloc[x, 1])
    return(varlist)

In [165]:
ExtractFeatureSelection(mod.stages[-1].selectedFeatures, df2, "features")

,idx,name
0,0,OHE_EventID_12
1,1,OHE_EventID_10
2,2,OHE_EventID_7
3,3,OHE_EventID_13
4,4,OHE_EventID_4658
5,5,OHE_EventID_4103
6,6,OHE_EventID_4656
7,7,OHE_EventID_4690
8,8,OHE_EventID_4663
9,9,OHE_EventID_800


In [166]:
# Try again with more columns

In [167]:
# Select subset of columns with more than 1 AND less than 52 unique values -> top 20
columnList = list(pd_target_distict_count.loc[(pd_target_distict_count[0] > 1) & (pd_target_distict_count[0] < 52)].sort_values(by = 0, ascending = False).iloc[:20].index.values)
#columnList = ["EventID", "ProcessGuid"]

# one hot encoding and assembling
encoding_var = columnList
#num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='y')]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]
onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]
label_indexes = StringIndexer(inputCol = 'TP_Binary', outputCol = 'label', handleInvalid = 'keep')
assembler = VectorAssembler(inputCols = ['OHE_' + c for c in encoding_var], outputCol = "features")
selector = ChiSqSelector(numTopFeatures=50, featuresCol='features', outputCol="selectedFeatures", labelCol='label', selectorType='numTopFeatures', percentile=0.1, fpr=0.05, fdr=0.05, fwe=0.05)

In [168]:
pipe = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes, selector])

In [169]:
mod = pipe.fit(df_index)

In [170]:
df2 = mod.transform(df_index)

In [171]:
ExtractFeatureSelection(mod.stages[-1].selectedFeatures, df2, "features")

,idx,name
0,0,OHE_ProcessId_3852
1,1,OHE_ProcessId_5044
2,2,OHE_ProcessId_900
3,3,OHE_ProcessId_884
4,4,OHE_ProcessId_0xf0c
5,5,OHE_ProcessId_944
6,1107,OHE_ProcessGuid_{47ab858c-e374-5eac-d803-000000000400}
7,1108,OHE_ProcessGuid_{47ab858c-e6c0-5eac-7600-000000000500}
8,1757,OHE_ThreadID_4612
9,1758,OHE_ThreadID_4588


In [1]:
df_index

NameError: name 'df_index' is not defined